## Introduction

In this GitHub repo I maintain a data set on TV rating that is originally published by the Swiss broadcaster Schweizer Radio und Fernsehen (SRF). 

The SRF publishes the ratings of its TV channels SRF1 and SRFzwei on http://www.srf.ch/medien/publikumszahlen/ in PDF format (one PDF per day and channel). This makes it rather difficult to look at the program performance over time and/or to analyse the performance of certain shows.

To get around this difficulty, I scraped and extracted the data from the PDFs. This notebook documents how this was done.

## The elements

To get from the PDF files to the dataset, several steps need to be made. So, we will write some functions in Python to achive the following tasks:

1. I will generate the specific URLs corresponding to the date and channel we are interested in
2. To download the PDF file, we will make a HTTP GET request.
3. Then, we'll need to extract the text from the PDF and ...
4. to format the text (rows and columns) so that it can be written in csv-like format.
5. Finally, we will write the formatted text to a csv file.
6. Additionally, we will need some helper functions and
7. a wrapper function.

Most of these functions are really straight forward. The main work is down by `get_program_pdf()` (generating the urls) and `convert_pdf()` (formatting the data). All functions a wrapped by `get_program_data()`.

In [ ]:
# I'll use the following packages trying to limit it to built-in packages (except for PyPDF2)
# since later I'd like to run the script on a shared webhosting server

import csv
from datetime import datetime, timedelta
import io
import re
import requests
from PyPDF2 import PdfFileReader

## 1. Reconstructing URLs

The [SRF website](http://www.srf.ch/medien/publikumszahlen/) is Wordpress-based and uses a the common blog-like structure where the uploads a filed by year and month:

http://www.srf.ch/medien/wp-content/uploads/<mark>2017/12/SRF1_171205_DI</mark>.pdf<br />
http://www.srf.ch/medien/wp-content/uploads/2017/12/SRFzwei_171205_DI.pdf


The name of the PDF files is structured along the name of the channel (e.g. SRF1 or SRFzwei), the date (formatted like `%y%m%d`) and the German abbreviation of the weekday (e.g., DI for Tuesday).

Thus, in order to download the PDFs in an automated way, we first need to reconstruct the URL for each day for which we like to download the PDF.</p>

The components needed are the following:

* year: four digits (yyyy)
* month: two digits (mm)
* complete date: yymmdd (e.g., 171205)
* weekday: german abbreviation


In [ ]:
def get_program_pdf(date, channel):
    """Constructs a URL for corresponding date and downloads the PDF file.
    
    Parameters:
        date (string): '%Y-%m-%d'
        channel (string): 'SRF1' or 'SRFzwei'
        
    returns a PDF file (file-like object)
    """
    date = datetime.strptime(date, '%Y-%m-%d')
    
    # translation of weekday number to German weekday abbreviation
    weekday_strings = {0: 'MO', 1: 'DI', 2: 'MI', 3: 'DO', 4: 'FR', 5: 'SA', 6: 'SO'}
    
    year = date.year        # year of the upload
    m = date.strftime('%m') # month of the upload
    wd_s = weekday_strings[date.weekday()]
    short_date = date.strftime('%y%m%d')
    
    url = 'http://www.srf.ch/medien/wp-content/uploads/{0}/{1}/{2}_{3}_{4}.pdf'.format(year, m, channel, short_date, wd_s)

    pdf = get_pdf(url)
    
    if pdf != None:
        return pdf
    else:
        # the PDF files are uploaded with some delay, thus, the ratings of the last day(s) of the month are
        # eventually uploaded in the following month
        next_m = date + timedelta(days=5)
        m = next_m.strftime('%m')
        year = next_m.year
        url = 'http://www.srf.ch/medien/wp-content/uploads/{0}/{1}/{2}_{3}_{4}.pdf'.format(year, m, channel, short_date, wd_s)
        pdf = get_pdf(url)
        if pdf != None:
            return pdf
        else:
            raise ValueError("URL doesn't exist.")

## 2. HTTP GET request

In [ ]:
def get_pdf(url):
    """Downloads the specified PDF file.
    
    Parameter:
        url (string): url
        
    returns file-like object or None
    """
    # Look like a humane
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:10.0) Gecko/20100101 Firefox/10.0'}
    
    rq = requests.get(url, headers=headers, timeout=5)
    if rq.status_code == 200:
        return io.BytesIO(rq.content)
    else:
        return None

## 3. Extracting the text data

In [ ]:
def extract_text(pdf):
    """Extracts the text of a PDF file.
    
    returns a string
    """
    pdf = PdfFileReader(pdf)
    page = pdf.getPage(0)
    return page.extractText()

## 4. Formatting the data

In [ ]:
def convert_pdf(pdf, channel):
    """Converts the extracted PDF content to csv file format.
    
    Parameters:
        pdf (io.ByteIO, file-like object): PDF file
        channel (string): name of the channel
        
    Returns a list of lists
    """
    txt = extract_text(pdf)
    
    # Splitting text into rows by \n (end of line) if certain conditions are met
    pat = re.compile('\\n(?=\d{2}\:\d{2}\\n\d{2}\:\d{2})|(?<=\.\d)\\n(?!\d+\\n)')
    rows = re.split(pat, txt)
    
    # Extract date from the first row
    date_pat = re.compile('\d{2}\.\d{2}\.\d{4}')
    ref_date = datetime.strptime(re.findall(date_pat, rows[0])[0], '%d.%m.%Y')
    
    # Split rows into cells
    table = [row.split('\n') for row in rows[1:-1]]
    
    # The PDF columns Beginn and Ende contain only time
    # So, we creating lists of datetime object to add the full date to each row
    # using the reference date extracted from the first row
    begin = [datetime.strptime(row[0], '%H:%M').time() for row in table]
    end = [datetime.strptime(row[1], '%H:%M').time() for row in table]
    
    begin_fulldate = create_date(ref_date, begin)
    end_fulldate = create_date(ref_date, end)
    
    # Ensure that the rows have the same length (9 columns)
    # Rows with length 8 do not have a Untertitel in which case we'll insert an empty string ('')
    # The last row contains meta information that will be cut off
    new_table = []
    for i, row in enumerate(table):
        if len(row) == 8:
            row.insert(4, '')
        elif len(row) > 9:
            row = row[:9]
        row[0] = begin_fulldate[i]
        row[1] = end_fulldate[i]
        row.append(channel)
        new_table.append(row)
    
    return new_table

## 5. Writing the data to csv file

In [ ]:
def write_csv(data, file_name):
    """Writes a csv file.
    
    Parameters:
        data (list, array-like object): rows of the csv file
        file:name (string): file name of the csv
    """
    with open(file_name, 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

## 6. Helper functions

In [ ]:
def create_date(d, ts):
    """Helper function to combine reference date and time.
       
   Parameters:
       d (datetime.date object): reference date
       ts (list, array-like object): datetime.time objects
    
    returns a list of datetime objects
    """
    k = len(ts)
    i = 1
    dt_series = []
    while i <= k:
        try:
            if ts[i-1] <= ts[i]:
                dt_series.append(datetime.combine(d, ts[i-1]))
            else:
                dt_series.append(datetime.combine(d, ts[i-1]))
                d = d + timedelta(days=1)
        except IndexError:
            dt_series.append(datetime.combine(d, ts[i-1])) # last row in the Series will be combined with date
        i += 1
    return [date.strftime('%Y%m%d %H:%M:%S') for date in dt_series]    
    
    
def date_generator(lag=4):
    """Generates a date.
    
    Parameters:
        lag (integer): time lag in days
        
    returns a date string
    """
    date = datetime.today() - timedelta(days=lag)
    return date.strftime('%Y-%m-%d')

## 7. Main wrapper function

In [ ]:
def get_program_data(date, channel, csvfile_name):
    pdf = get_program_pdf(date, channel=channel)
    data = convert_pdf(pdf, channel)
    write_csv(data, file_name=csvfile_name)

The script is executed like this:

`get_program_data(date="2019-02-12", channel="SRF1", csvfile_name="srf_program_ratings.csv")`